In [1]:
 # In Terminal, "pip install ibis-framework[duckdb] pyjanitor"
import pandas as pd
import ibis
from ibis import selectors as s
from ibis import _
ibis.options.interactive = True

In [2]:
# Path
from pathlib import Path
path = Path("/home/uiv17345/datasets/home-dataset/jupyterlab/PnL")
data_path = path / "data" / "SAP YGL4"
output_path = path / "output"

In [3]:
data_path

PosixPath('/home/uiv17345/datasets/home-dataset/jupyterlab/PnL/data/SAP YGL4')

### Input data: List of multiple text files

In [4]:
# Input data: List of multiple text files
dat_files = [file for file in data_path.iterdir() if file.is_file() and file.suffix == ".dat"]
dat_files

[PosixPath('/home/uiv17345/datasets/home-dataset/jupyterlab/PnL/data/SAP YGL4/EBIT.dat'),
 PosixPath('/home/uiv17345/datasets/home-dataset/jupyterlab/PnL/data/SAP YGL4/FX trading.dat'),
 PosixPath('/home/uiv17345/datasets/home-dataset/jupyterlab/PnL/data/SAP YGL4/Sales.dat'),
 PosixPath('/home/uiv17345/datasets/home-dataset/jupyterlab/PnL/data/SAP YGL4/Variable Costs over STD.dat')]

In [5]:
def read_multiple_files(list_of_files):
    dataframes = [
        pd.read_csv(
            file,
            sep="\t",
        )
        for file in list_of_files
    ]
    
    # Add a new column with filename to each DataFrame
    for i, df in enumerate(dataframes):
        df["source"] = list_of_files[i].stem

    # Merge the list of DataFrames into a single DataFrame
    df = pd.concat(dataframes)

    # reorder columns
    df = df[["source"] + [col for col in df.columns if col not in ["source"]]]
    
    return df

In [6]:
def change_column_names(df):
    return df.rename(
        columns={
            # "source": "Items",
            "OneGL B/S + P/L": "OneGL",
            "01": "Jan", "02": "Feb", "03": "Mar",
            "04": "Apr", "05": "May", "06": "Jun",
            "07": "Jul", "08": "Aug", "09": "Sep",
            "10": "Oct", "11": "Nov", "12": "Dec"
        }
    )

In [7]:
df = read_multiple_files(dat_files)
df = change_column_names(df)
df.head(3)

,source,OneGL,0-2,1-2,CF,Jan,Feb,Mar,Apr,May,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,EBIT,50802-018 P SC Mechatronic Sensors Sejong,208503808,208503808,0.0,-25508605.0,2.340124e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EBIT,50803-003 E CT Engine Controls Icheon,0,0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EBIT,50803-004 E Central Icheon NPF,1654517315,1654517315,0.0,-137988495.0,1.792506e+09,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data wrangling

In [8]:
t = ibis.memtable(df, name="t")
t = t.select("source", "OneGL", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")
t = (t
        .mutate(source = _.source.replace(".dat", ""))
        .mutate(PrCr = _.OneGL.re_extract(r"([0-9\-]{8,9})", 1))
        .mutate(s.across(s.numeric(), _ / -10**6))
        .filter(_.PrCr != "")  # instead of .dropna("PrCr")
        .rename({'Items':"source"})
    )
t

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ Items  ┃ OneGL                                          ┃ Jan         ┃ Feb          ┃ Mar     ┃ Apr     ┃ May     ┃ Jun     ┃ Jul     ┃ Aug     ┃ Sep     ┃ Oct     ┃ Nov     ┃ Dec     ┃ PrCr      ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ string                                         │ float64     │ float64      │ float64 │ float64 │ float64 │ float64 │ float64 │ float64 │ float64 │ float64 │ float64 │ float64 │ string    │
├────────┼────────────────────────────────────────────────┼─────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼───────────┤
│ EBIT   │ 50802-018  P SC Mechatronic Sensors Sejong     │   25.508605 │  -234.012413 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50802-018 │
│ EBIT   │ 50803-003  E CT Engine Controls Icheon         │   -0.000000 │    -0.000000 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-003 │
│ EBIT   │ 50803-004  E Central Icheon NPF                │  137.988495 │ -1792.505810 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-004 │
│ EBIT   │ 50803-009  P AC Engine Actuators Icheon        │  175.435656 │  -949.955687 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-009 │
│ EBIT   │ 50803-010  E CT Drivetrain Controls Icheon     │ 1360.279978 │   909.398731 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-010 │
│ EBIT   │ 50803-025  E CT Engine Controls Icheon NPF     │   -0.000000 │    -0.000000 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-025 │
│ EBIT   │ 50803-026  P HT Hydraulics Icheon              │  455.111380 │ -1113.243374 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-026 │
│ EBIT   │ 50803-028  E CT Drivetrain Controls Icheon NPF │   -0.000000 │    -0.000000 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-028 │
│ EBIT   │ 50803-034  P SC Mechatronic Sensors Icheon     │  -93.457999 │   144.088114 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-034 │
│ EBIT   │ 50803-04   E CT eMotor Controls Icheon NPF     │   44.954346 │  1538.675937 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │    -0.0 │ 50803-04  │
│ …      │ …                                              │           … │            … │       … │       … │       … │       … │       … │       … │       … │       … │       … │       … │ …         │
└────────┴────────────────────────────────────────────────┴─────────────┴──────────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴───────────┘

In [9]:
df = t.to_pandas()
df.to_csv(output_path / "SAP YGL4 P&L.csv", index=False)